In [1]:
import sqlite3

import pandas as pd
import xgboost as xgb
from sklearn.linear_model import ElasticNet
from sklearn.metrics import explained_variance_score

In [2]:
connection = sqlite3.connect('stonks.db')
query = '''
    SELECT "index", "date", "time_of_day", "price"
    FROM prices
    WHERE "hemisphere" == "NH" AND "time_of_day" != "purchase"
'''

with connection:
    df = pd.read_sql_query(query, con=connection, index_col='index')

In [3]:
regr = ElasticNet()

In [4]:
# only dates with an 'am' and 'pm' entry can be used for training
usable_dates = [date for date, count in df.groupby('date').count()['price'].items() if count == 2 ]

In [5]:
df = df[df['date'].isin(usable_dates)]

In [6]:
X = df[df['time_of_day'] == 'am']['price'].values

In [7]:
y = df[df['time_of_day'] == 'pm']['price'].values

In [8]:
assert len(X) == len(y)

In [9]:
regr.fit(X.reshape(-1, 1), y)

ElasticNet()

In [10]:
explained_variance_score(y_true=y, y_pred=regr.predict(X.reshape(-1, 1)))

0.6951698509682647

In [11]:
regr.predict([[84]])

array([85.88728667])